# AI Fitness Trainer — Thesis Analysis

**Master's Thesis:** *Development of an Intelligent System for Monitoring the Performance of Physical Exercises Using Computer Vision and Deep Learning*

This notebook generates all quantitative results and figures for the thesis:
1. Synthetic dataset statistics
2. Model training curves
3. Per-exercise evaluation (MAE, Pearson, ICC)
4. Three-way comparison: Rule-based vs DL vs Hybrid
5. Thesis figures (publication-ready)


In [ ]:
import sys
from pathlib import Path

# Add project root to path
ROOT = Path('..').resolve()
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import tensorflow as tf

from config import EXERCISES, SCORE_DIMS, ANGLE_NAMES

sns.set_theme(style='whitegrid', palette='muted', font_scale=1.2)
plt.rcParams['figure.dpi'] = 120
print('TensorFlow', tf.__version__)
print('NumPy', np.__version__)


---
## 1  Synthetic Dataset Statistics

In [ ]:
from data.dataset_loader import load_synthetic

fig, axes = plt.subplots(2, 3, figsize=(14, 8))
fig.suptitle('Synthetic Dataset — Score Distributions', fontsize=14, fontweight='bold')

colors = sns.color_palette('muted', len(EXERCISES))

for i, dim in enumerate(SCORE_DIMS):
    ax = axes.flat[i]
    for ex, color in zip(EXERCISES, colors):
        try:
            _, scores = load_synthetic(ex)
            ax.hist(scores[:, i] * 100, bins=40, alpha=0.5,
                    label=ex.replace('_', ' '), color=color, density=True)
        except FileNotFoundError:
            pass
    ax.set_title(dim, fontsize=11)
    ax.set_xlabel('Score (0–100)')
    ax.set_ylabel('Density')
    if i == 0:
        ax.legend(fontsize=8)

plt.tight_layout()
plt.savefig(ROOT / 'evaluation/figures/dataset_score_distributions.png',
            dpi=150, bbox_inches='tight')
plt.show()


In [ ]:
# Summary statistics table
rows = []
for ex in EXERCISES:
    try:
        seqs, scores = load_synthetic(ex)
        s100 = scores * 100
        row = {'exercise': ex, 'n_samples': len(seqs)}
        for i, dim in enumerate(SCORE_DIMS):
            row[f'{dim}_mean'] = round(s100[:, i].mean(), 1)
            row[f'{dim}_std']  = round(s100[:, i].std(),  1)
        rows.append(row)
    except FileNotFoundError:
        rows.append({'exercise': ex, 'n_samples': 0, 'note': 'NOT FOUND'})

df_stats = pd.DataFrame(rows)
print('Synthetic dataset summary')
display(df_stats[['exercise', 'n_samples'] +
                 [f'{d}_mean' for d in SCORE_DIMS]])


---
## 2  Sample Rep Trajectories

In [ ]:
from data.dataset_loader import load_synthetic

fig, axes = plt.subplots(1, len(EXERCISES), figsize=(16, 4), sharey=True)
fig.suptitle('Sample Rep Trajectories (one per exercise)', fontsize=13)

for ax, ex in zip(axes, EXERCISES):
    try:
        seqs, scores = load_synthetic(ex)
        # Pick a high-quality rep (overall score > 80)
        good_idx = np.where(scores[:, 5] * 100 > 80)[0]
        idx = good_idx[0] if len(good_idx) > 0 else 0
        seq = seqs[idx]
        for j, name in enumerate(ANGLE_NAMES):
            ax.plot(seq[:, j], linewidth=1.1, label=name)
        ax.set_title(ex.replace('_', ' ').title(), fontsize=10)
        ax.set_xlabel('Frame')
    except FileNotFoundError:
        ax.set_title(f'{ex}\n(no data)')

axes[0].set_ylabel('Angle (°)')
axes[0].legend(fontsize=6, ncol=2)
plt.tight_layout()
plt.savefig(ROOT / 'evaluation/figures/sample_trajectories.png',
            dpi=150, bbox_inches='tight')
plt.show()


---
## 3  Training History

In [ ]:
LOGS_DIR = ROOT / 'models' / 'logs'

fig, axes = plt.subplots(1, len(EXERCISES), figsize=(16, 4), sharey=True)
fig.suptitle('Training Curves — Phase 1 (Pre-train on Synthetic Data)', fontsize=13)

for ax, ex in zip(axes, EXERCISES):
    log_path = LOGS_DIR / f'{ex}_phase1_history.csv'
    if not log_path.exists():
        ax.set_title(f'{ex}\n(not trained)')
        continue
    hist = pd.read_csv(log_path)
    ax.plot(hist['epoch'], hist['mae'] * 100,     label='Train MAE', linewidth=1.5)
    ax.plot(hist['epoch'], hist['val_mae'] * 100, label='Val MAE',   linewidth=1.5, linestyle='--')
    ax.axhline(15, color='red', linestyle=':', linewidth=1, label='Target < 15')
    ax.set_title(ex.replace('_', ' ').title(), fontsize=10)
    ax.set_xlabel('Epoch')

axes[0].set_ylabel('MAE (0–100 scale)')
axes[0].legend(fontsize=8)
plt.tight_layout()
plt.savefig(ROOT / 'evaluation/figures/training_curves_phase1.png',
            dpi=150, bbox_inches='tight')
plt.show()


---
## 4  Model Evaluation (MAE · Pearson · ICC)

In [ ]:
from evaluation.evaluate_model import evaluate_all

df_eval = evaluate_all(phase=3, verbose=True)
display(df_eval.round(3))


In [ ]:
# Heat map: MAE per exercise × dimension
if not df_eval.empty:
    pivot = df_eval.pivot(index='exercise', columns='dimension', values='mae')[SCORE_DIMS]

    fig, ax = plt.subplots(figsize=(10, 4))
    sns.heatmap(pivot, annot=True, fmt='.1f', cmap='YlOrRd_r',
                vmin=0, vmax=20, linewidths=0.5, ax=ax)
    ax.set_title('MAE Heatmap — DL Model (Phase 3)', fontsize=13)
    ax.set_xlabel('Score Dimension')
    ax.set_ylabel('Exercise')
    plt.tight_layout()
    plt.savefig(ROOT / 'evaluation/figures/mae_heatmap.png',
                dpi=150, bbox_inches='tight')
    plt.show()


---
## 5  Three-Way Comparison: Rule-based vs DL vs Hybrid

In [ ]:
from evaluation.compare_approaches import compare_all

df_cmp = compare_all(phase=3)
display(df_cmp.round(3))


In [ ]:
# Publication-ready comparison bar chart
from evaluation.visualize_predictions import hybrid_comparison_bar

fig = hybrid_comparison_bar(phase=3, save=True)
plt.show()


---
## 6  Predicted vs Actual Scatter Plots

In [ ]:
from evaluation.visualize_predictions import scatter_predicted_vs_actual
from models.form_scorer_model import build_form_scorer

MODELS_DIR = ROOT / 'models' / 'saved'

for ex in EXERCISES:
    model_path = MODELS_DIR / f'{ex}_phase3_final.keras'
    if not model_path.exists():
        print(f'[SKIP] {ex}')
        continue
    model = tf.keras.models.load_model(str(model_path))
    fig   = scatter_predicted_vs_actual(ex, model, save=True)
    plt.show()


---
## 7  Model Architecture Summary

In [ ]:
from models.form_scorer_model import build_form_scorer, compile_model

model = build_form_scorer()
model.summary()
print(f'\nTotal trainable parameters: {model.count_params():,}')


---
## 8  TFLite Export Verification

In [ ]:
from export.convert_tflite import DEFAULT_OUT

tflite_dir = ROOT / 'export' / 'tflite'
rows = []
for ex in EXERCISES:
    path = tflite_dir / f'form_scorer_{ex}_v1.tflite'
    if path.exists():
        rows.append({'exercise': ex, 'size_KB': round(path.stat().st_size / 1024, 1),
                     'status': '✓'})
    else:
        rows.append({'exercise': ex, 'size_KB': '-', 'status': 'not yet exported'})

df_tflite = pd.DataFrame(rows)
print('TFLite model files:')
display(df_tflite)
print(f'\nTotal: {df_tflite[df_tflite["size_KB"] != "-"]["size_KB"].astype(float).sum():.1f} KB')
